In [3]:
import pandas as pd
import json
import numpy as np

In [11]:
data_file = open('data/CSE151B_groupchat_3mo.json')
json_data = json.load(data_file)
json_data['messages'][1]['content'].encode('latin1').decode('utf-8')

"Can one of you guys download your facebook messenger data and upload the json file for this groupchat to the github repo? I can't download the 3 month one for some reason"

In [16]:
junk_messages = ['reacted', 'X:1759 T:F\"ur Elise T:Bagatelle', 'Counter({50:']

In [17]:
with open('data/cleaned.txt', 'w+') as f:
    for d in json_data['messages'][::-1]:
        if 'content' in d:
            message = d['content']
            if np.all([junk not in message for junk in junk_messages]):
                cleaned = message.replace('\n', ' ').encode('latin1').decode('utf-8')
                cleaned = '<start> ' + cleaned + ' <end> '
                f.write(cleaned)

In [14]:
with open('data/cleaned_with_name.txt', 'w+') as f:
    for d in json_data['messages'][::-1]:
        if 'content' in d:
            message = d['content']
            if np.all([junk not in message for junk in junk_messages]):
                sender = d["sender_name"].split(' ')[0].replace('Kong', 'Winfrey')
                cleaned = message.replace('\n', ' ').encode('latin1').decode('utf-8')
                cleaned = f'<start> {sender}: ' + cleaned + ' <end> '
                f.write(cleaned)